In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
import sys
import numpy as np 
import pandas as pd 
from datetime import datetime
import time
import random
from tqdm.autonotebook import tqdm

#CV
import cv2

#Glob
from glob import glob

<ipython-input-2-ff558d84ea22>:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
path = '/content/drive/MyDrive/data/programmers_ai/yujatea/comp_3/dataset/train/'

train = pd.read_csv(os.path.join(path, 'train_output.csv'))
train.head()

,filename,label
0,1,1 0.438281 0.529551 0.814063 0.704492
1,2,3 0.541016 0.524324 0.849219 0.672973
2,3,2 0.492500 0.517045 0.695000 0.725000
3,4,2 0.585156 0.525000 0.692187 0.785135
4,5,3 0.385547 0.645270 0.614844 0.698649\n3 0.559...


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1204 entries, 0 to 1203
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  1204 non-null   int64 
 1   label     1204 non-null   object
dtypes: int64(1), object(1)
memory usage: 18.9+ KB


In [5]:
def change_file_name(df):
  f_name_list = []
  for row in tqdm(df.iterrows()):
    f_name = row[1]['filename']
    f_len = len(str(f_name))
    if f_len == 1:
      result = f'000{f_name}.jpg'
    elif f_len == 2:
      result = f'00{f_name}.jpg'
    elif f_len == 3:
      result = f'0{f_name}.jpg'
    elif f_len == 4:
      result = f'{f_name}.jpg'
  
    f_name_list.append(result)
  df['filename'] = f_name_list

In [6]:
change_file_name(train)

0it [00:00, ?it/s]

In [7]:
train.head()

,filename,label
0,0001.jpg,1 0.438281 0.529551 0.814063 0.704492
1,0002.jpg,3 0.541016 0.524324 0.849219 0.672973
2,0003.jpg,2 0.492500 0.517045 0.695000 0.725000
3,0004.jpg,2 0.585156 0.525000 0.692187 0.785135
4,0005.jpg,3 0.385547 0.645270 0.614844 0.698649\n3 0.559...


In [8]:
# DataFrame 재생성
def label_extractor(df, path):
  temp = {
      'filename': [],
      'width': [],
      'height': [],
      'x': [],
      'y': [],
      'w': [],
      'h': [],
      'label': []
  }
  for row in tqdm(df.iterrows()):
    f_name = row[1]['filename']
    img = cv2.imread(os.path.join(path, f_name))
    tmp = row[1]['label'].split('\n') # ['label x_c, y_c, w, h' ...], 비율로 되있음
    for item in tmp:
      info = item.split()
      label = int(info[0])
      x, y, w, h = map(float, info[1:])
      temp['filename'].append(f_name)
      temp['width'].append(img.shape[1])
      temp['height'].append(img.shape[0])
      temp['x'].append(round((x- w/2)*img.shape[1], 1))
      temp['y'].append(round((y-h/2)*img.shape[0], 1))
      temp['w'].append(round(w * img.shape[1], 1))
      temp['h'].append(round(h * img.shape[0], 1))
      temp['label'].append(label)

  new_df = pd.DataFrame(temp)
  return new_df

In [9]:
my_train = label_extractor(train, path)
my_train

0it [00:00, ?it/s]

,filename,width,height,x,y,w,h,label
0,0001.jpg,640,423,20.0,75.0,521.0,298.0,1
1,0002.jpg,1992,1328,231.9,249.4,1691.6,893.7,3
2,0003.jpg,600,440,87.0,68.0,417.0,319.0,2
3,0004.jpg,1600,1065,382.5,141.0,1107.5,836.2,2
4,0005.jpg,800,450,62.5,133.2,491.9,314.4,3
...,...,...,...,...,...,...,...,...
2122,1200.jpg,500,333,40.0,138.0,126.0,165.0,1
2123,1201.jpg,800,411,201.0,140.0,494.0,265.0,0
2124,1202.jpg,1920,1080,400.5,55.5,1110.0,858.2,1
2125,1203.jpg,220,157,12.0,28.0,204.0,129.0,0


In [10]:
my_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2127 entries, 0 to 2126
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   filename  2127 non-null   object 
 1   width     2127 non-null   int64  
 2   height    2127 non-null   int64  
 3   x         2127 non-null   float64
 4   y         2127 non-null   float64
 5   w         2127 non-null   float64
 6   h         2127 non-null   float64
 7   label     2127 non-null   int64  
dtypes: float64(4), int64(3), object(1)
memory usage: 133.1+ KB


In [11]:
my_train.to_csv(os.path.join('/content/drive/MyDrive/data/programmers_ai/yujatea/comp_3/dataset/', 'my_train.csv'), index=False)

In [12]:
test = pd.read_csv(os.path.join('/content/drive/MyDrive/data/programmers_ai/yujatea/comp_3/dataset/test/', 'test_output.csv'))

In [13]:
test.head()

,filename,label
0,1205,1 0.662500 0.482394 0.606250 0.607981\n1 0.207...
1,1206,3 0.703906 0.629977 0.367188 0.707260\n3 0.331...
2,1207,2 0.511250 0.540338 0.832500 0.615385
3,1208,0 0.551562 0.545946 0.484375 0.494595
4,1209,3 0.429000 0.564000 0.702000 0.749333\n3 0.402...


In [14]:
change_file_name(test)

0it [00:00, ?it/s]

In [15]:
t_path = '/content/drive/MyDrive/data/programmers_ai/yujatea/comp_3/dataset/test/'
my_test = label_extractor(test, t_path)

0it [00:00, ?it/s]

In [16]:
my_test.to_csv(os.path.join('/content/drive/MyDrive/data/programmers_ai/yujatea/comp_3/dataset/', 'my_test.csv'), index=False)